In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name() 
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-12-12 09:28:15.409645: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-12-12 09:28:15.409659: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-12-12 09:28:15.409664: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-12-12 09:28:15.410235: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-12 09:28:15.410705: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [19]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler,SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup

In [20]:
device = torch.device("mps")
print(device)

mps


In [21]:
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt